In [ ]:
#As imagens normalmente tem 1 ou 3 canais, mas essa simulação só usa 2
#Então, para não aparecer um monte de avisos, coloquei essa parte
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")

In [ ]:
import astropy.io.fits as fits
import numpy as np
import matplotlib.pyplot as plt
from astropy.visualization import simple_norm  # Import simple_norm

# Caminho para o arquivo FITS
file_path = r"C:\Users\stefa\OneDrive\Documentos\NATALI TCC\hlsp_deepmerge_hst_acs-wfc3_illustris-z2_f814w-f160w_v1_sim-pristine.fits"

# Abre o arquivo FITS
hdulist = fits.open(file_path)

# Extrai as imagens do HDU[0] e imprime o shape
images = hdulist[0].data
print(f"Shape das imagens: {images.shape}")  
# valor esperado: (15426, 2, 75, 75)

# Selecionar 3 imagens aleatórias de cada filtro [ F160W (índice=1) e F814W (índice=0)]

example_ids = np.random.choice(hdulist[1].data.shape[0], 2)
print(example_ids)

examples_f160w = [hdulist[0].data[j, 1, :, :] for j in example_ids]
examples_f814w = [hdulist[0].data[j, 0, :, :] for j in example_ids]

# Inicializar a figura
fig = plt.figure(figsize=(16, 16))  # Ajuste a altura para separar as imagens

# Loop pelas imagens selecionadas aleatoriamente e plotar com rótulos
for i, (image_f160w, image_f814w) in enumerate(zip(examples_f160w, examples_f814w)):
    # F160W
    ax1 = fig.add_subplot(8, 4, i * 2 + 1)  # Ajuste o layout para 8 linhas e 4 colunas
    norm1 = simple_norm(image_f160w, 'log', max_percent=99.75)
    ax1.imshow(image_f160w, aspect='equal', cmap='binary_r', norm=norm1)
    ax1.set_title('F160W') 
    ax1.axis('off')

    # F814W
    ax2 = fig.add_subplot(8, 4, i * 2 + 2)  # Ajuste o layout para 8 linhas e 4 colunas
    norm2 = simple_norm(image_f814w, 'log', max_percent=99.75)
    ax2.imshow(image_f814w, aspect='equal', cmap='binary_r', norm=norm2)
    ax2.set_title('F814W')  
    ax2.axis('off')

plt.tight_layout()  # Ajuste o layout para evitar sobreposição dos títulos
plt.show()

# Salve todas as imagens, de ambos os filtros, na variável X
X = images
print(X.shape)

hdulist.close()

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Transpõe para o formato esperado pelo ImageDataGenerator
X = X.transpose(0, 2, 3, 1) 
print("Após transposição:", X.shape)

# Converte os dados para o tipo float32
X = X.astype(np.float32)
print("Após conversão para float32:", X.shape)

# Normaliza os dados originais
X = X / 255.0  # Normalizando para o intervalo [0, 1]
print("Após normalização:", X.shape)

#Criação do ImageDataGenerator (sem normalização, pois já está normalizado)
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotação de até 20 graus
    width_shift_range=0.1,  # Deslocamento horizontal de até 10% da largura
    height_shift_range=0.1,  # Deslocamento vertical de até 10% da altura
    shear_range=0.1,  # Distorção por cisalhamento de até 10%
    zoom_range=0.1,  # Zoom de até 10%
    horizontal_flip=True,  # Reflexão horizontal com 50% de probabilidade
    fill_mode='nearest'  # Preenchimento de pixels faltantes
)

# Geração de imagens aumentadas em batches e salvamento em arquivo .fits
num_augmented_images = int(15426 * 1.5)  # Aumenta em 50% (1.5 = 1 + 0.5)
X_augmented = []
for batch_index, batch in enumerate(datagen.flow(X, batch_size=32, shuffle=True)):
    # Reorganiza as dimensões
    batch = batch.reshape(-1, 2, 75, 75)
    X_augmented.extend(batch)
    # Verifica se o número de imagens aumentadas atingiu o limite
    if len(X_augmented) >= num_augmented_images:
        break

# Converte as listas para arrays NumPy
X_augmented = np.array(X_augmented)[:num_augmented_images]
print("Após aumento de dados:", X_augmented.shape)

# Salva como um arquivo
np.save("X_augmented.npy", X_augmented)

In [ ]:
import astropy.io.fits as fits
import numpy as np

# Caminho para o arquivo FITS
file_path = r"C:\Users\stefa\OneDrive\Documentos\NATALI TCC\hlsp_deepmerge_hst_acs-wfc3_illustris-z2_f814w-f160w_v1_sim-pristine.fits"

# Abre o arquivo FITS
hdulist = fits.open(file_path)

#Extrai os labels do HDU[1] e imprime o shape
labels = hdulist[1].data
print(f"Shape dos labels: {labels.shape}")  
# valor esperado: (15426,)

# Salva os labels na variável Y
Y = labels

# Converte as listas para arrays NumPy e printe o shape
Y = np.array(Y)
print(Y.shape)

# Salva o array Y em um arquivo .npy
np.save("Y.npy", Y)

# Fecha o arquivo FITS
hdulist.close()

In [ ]:
import numpy as np
from astropy.io import fits

# Caminho do arquivo FITS
file_fits = "C:\\Users\\stefa\\OneDrive\\Documentos\\NATALI TCC\\hlsp_deepmerge_hst_acs-wfc3_illustris-z2_f814w-f160w_v1_sim-pristine.fits"

# Abrir o arquivo FITS
hdulist = fits.open(file_fits)

# Extrair os labels
data_hdu1 = hdulist[1].data

# Criar as listas de merge e non-merge
list_of_mergers = np.where(data_hdu1 == 1.)[0]
list_of_nonmergers = np.where(data_hdu1 == 0.)[0]

# Salvar os arrays em arquivos .npy
np.save("mergers.npy", list_of_mergers)
np.save("nonmergers.npy", list_of_nonmergers)

# Fechar o arquivo FITS
hdulist.close()

print("Arrays salvos em mergers.npy e nonmergers.npy.")

In [ ]:
import numpy as np
import random

# Carrega os arrays X_augmented e Y
X_augmented = np.load("X_augmented.npy")
Y = np.load("Y.npy")
#valor esperado: X_augmented = (23139, 75, 75, 2) e Y=(15426,)

# Verifica os shapes
print(f"Shape de X_augmented: {X_augmented.shape}") 
print(f"Shape de Y: {Y.shape}")

# Seleciona aleatoriamente 15426 imagens de X_augmented
num_imagens = len(Y)  # Usar o tamanho de Y para garantir correspondência
indices_aleatorios = random.sample(range(len(X_augmented)), num_imagens)
X_augmented_aleatorio = X_augmented[indices_aleatorios]

# Cria um novo array com as imagens e labels
data = np.zeros((num_imagens, *X_augmented_aleatorio.shape[1:]), dtype=X_augmented.dtype)
data[:, :] = X_augmented_aleatorio

# Converta os labels para o tipo de dados correto
Y_type = X_augmented.dtype  # Obter o tipo de dados das imagens
Y = Y.astype(Y_type)  # Converta os labels para o tipo de dados das imagens

data[:, -1] = Y  # Adiciona os labels na última coluna

# Salva o array em data.npy
np.save("data.npy", data)

print("Arquivo salvo em data.npy")

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Carrega o array concatenado
data = np.load("data.npy")

# Extrai as imagens e os labels
X = data  # X_augmented já possui a forma correta das imagens
Y = data[:, -1]  # Assumindo que o label é a última coluna

# Divisão dos dados em treino, teste e validação
random_seed = 42
X_train, X_rem, Y_train, Y_rem = train_test_split(X, Y, test_size=0.3, random_state=random_seed)
X_test, X_val, Y_test, Y_val = train_test_split(X_rem, Y_rem, test_size=1/3, random_state=random_seed)

# Imprime o shape dos testes, treinos e validações de X e de Y
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)
print("X_val shape:", X_val.shape)
print("Y_val shape:", Y_val.shape)

# Salva os dados como arquivos .npy
np.save("X_train.npy", X_train)
np.save("Y_train.npy", Y_train)
np.save("X_test.npy", X_test)
np.save("Y_test.npy", Y_test)
np.save("X_val.npy", X_val)
np.save("Y_val.npy", Y_val)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, RMSprop
import numpy as np
import json
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

In [ ]:
# Define a camada de entrada (com 2 canais)
input_tensor = layers.Input(shape=(75, 75, 2)) 

In [ ]:
# Camadas convolucionais
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001))(input_tensor)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = layers.Dropout(0.2)(x)

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = layers.Dropout(0.3)(x)

x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001))(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
x = layers.Dropout(0.4)(x)

# Camadas densas
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001))(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l1_l2(l1=0.0001, l2=0.0001))(x)
x = layers.Dropout(0.5)(x)

# Camada de saída
output_tensor = layers.Dense(1, activation='sigmoid')(x)

In [ ]:
# Create the model
model = Model(inputs=input_tensor, outputs=output_tensor)

In [ ]:
# Parâmetros de compilação
optimizer = Adam(learning_rate=0.0001)
metrics = ['accuracy']
loss = 'binary_crossentropy'

In [ ]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# Caminho para salvar checkpoints
checkpoint_filepath = 'melhor_modelo_pesos.weights.keras'

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    verbose=1,
    save_best_only=True
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='min',
    verbose=1,
    patience=30, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=30,
    min_lr=1e-05
)

# Combine callbacks into a list:
callbacks = [checkpoint, early_stopping, reduce_lr]

In [ ]:
# Carrega os dados dos arquivos .npy
X_train = np.load("X_train.npy")
Y_train = np.load("Y_train.npy")
X_test = np.load("X_test.npy")
Y_test = np.load("Y_test.npy")
X_val = np.load("X_val.npy")
Y_val = np.load("Y_val.npy")

In [ ]:
# Treinar o modelo
history = model.fit(
    x=X_train,
    y=Y_train,
    batch_size=32,
    epochs=100,
    validation_data=(X_val, Y_val),
    shuffle=True,
    verbose=1,
    callbacks=callbacks
)

In [ ]:
# Salvar histórico, pesos e modelo
with open("history.json", "w") as f:
    json.dump(history.history, f)

model.save_weights("pesos_modelo.weights.h5")
model.save("modelo.keras")

# Carregar modelo salvo
loaded_model = load_model("modelo.keras")

In [ ]:
# Criar otimizador rmsprop (apenas uma vez)
rmsprop = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-07)

In [ ]:
# Compilar o modelo com a função de perda correta
loaded_model.compile(
    optimizer=rmsprop,
    loss='binary_crossentropy',  # Utilize 'binary_crossentropy' para classificação binária
    metrics=['accuracy']
)

In [ ]:
# Imprimir a acurácia do modelo carregado
accuracy = loaded_model.evaluate(X_test, Y_test, verbose=0)[1]  # Use os dados de teste aqui
print(f"Acurácia do modelo carregado: {accuracy * 100:.2f}%") 

In [ ]:
# Plotar histórico
with open("history.json", "r") as f:
    history_data = json.load(f)

loss = history_data['loss']
val_loss = history_data['val_loss']
acc = history_data['accuracy']
val_acc = history_data['val_accuracy']

epochs = list(range(len(loss)))
figsize=(6,4)
fig, axis1 = plt.subplots(figsize=figsize)

plot1_lacc = axis1.plot(epochs, acc, 'navy', label='accuracy')
plot1_val_lacc = axis1.plot(epochs, val_acc, 'deepskyblue', label="validation accuracy")
plot1_loss = axis1.plot(epochs, loss, 'red', label='loss')
plot1_val_loss = axis1.plot(epochs, val_loss, 'lightsalmon', label="validation loss")

plots = plot1_loss + plot1_val_loss + plot1_lacc + plot1_val_lacc
labs = [l.get_label() for l in plots]
axis1.set_xlabel('Epoch')
axis1.set_ylabel('Loss/Accuracy')
plt.title("Loss/Accuracy History (Pristine Images)")
plt.tight_layout()
axis1.legend(loc='center right')

plt.show()